<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/modelling/sk_RAG_jpmorgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
===================================================
Author: Sheldon Kemper
Role: Data Engineering Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/sheldon-kemper
Date: 2025-02-04
Version: 1.1

Description:
    This notebook implements a Retrieval-Augmented Generation (RAG) system using JP Morgan
    earnings transcripts as the source data. It builds on our existing data engineering pipeline
    by reading raw PDF files stored in Google Drive, extracting text using LangChain’s PyPDFLoader,
    and indexing the content with CHROMA and Sentence Transformer embeddings. A text generation model
    (Flan-T5) is then used to answer queries based on the retrieved context, and the functionality
    is wrapped as a tool for a LangChain agent to handle more complex interactions.

===================================================
"""


'\n===================================================\nAuthor: Sheldon Kemper\nRole: Data Engineering Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/sheldon-kemper\nDate: 2025-02-04\nVersion: 1.1\n\nDescription:\n    This notebook implements a Retrieval-Augmented Generation (RAG) system using JP Morgan\n    earnings transcripts as the source data. It builds on our existing data engineering pipeline\n    by reading raw PDF files stored in Google Drive, extracting text using LangChain’s PyPDFLoader,\n    and indexing the content with CHROMA and Sentence Transformer embeddings. A text generation model\n    (Flan-T5) is then used to answer queries based on the retrieved context, and the functionality\n    is wrapped as a tool for a LangChain agent to handle more complex interactions.\n\n===================================================\n'

## Set Up and Import Libraries

In [ ]:
!pip install -U langchain-community
!pip install pypdf
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18


## Import required libraries

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.agents import initialize_agent, Tool
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from google.colab import drive
from langchain.agents import initialize_agent, Tool

## Mount Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Set the raw directory (adjust if necessary)
raw_dir = "/content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan"

# List all PDF files in the raw directory
pdf_files = [os.path.join(raw_dir, file) for file in os.listdir(raw_dir) if file.endswith(".pdf")]
print(f"Found {len(pdf_files)} PDF files in {raw_dir}")


## Load and Process the PDFs

In [ ]:
# Initialize an empty list for documents
documents = []

# Loop over each PDF file and load its content, skipping the first chunk which is likely a header.
for pdf_file in pdf_files:
    try:
        loader = PyPDFLoader(pdf_file)
        docs = loader.load()
        # Option 1: Skip the first chunk if it is a header page
        if docs and "JPMorgan Chase" in docs[0].page_content and "Earnings Call Transcript" in docs[0].page_content:
            docs = docs[1:]
        documents.extend(docs)
        print(f"Loaded {len(docs)} chunks from {os.path.basename(pdf_file)} (header skipped if detected)")
    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")




## Initialize Sentence Transformer embeddings

In [ ]:
# Initialize Sentence Transformer embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

## Create a Chroma vector store from the loaded documents

In [ ]:
# Create a Chroma vector store from the loaded documents.
vectorstore = Chroma.from_documents(documents, embeddings, collection_name="jpm_transcripts")
print("Chroma vector store created from the JP Morgan transcripts.")

## Create a Chroma vector store from the loaded documents

In [ ]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents, embeddings, collection_name="jpm_transcripts")
print("Chroma vector store created from the JP Morgan transcripts.")


## Load the Flan-T5 model for text generation using Hugging Face's transformers pipeline

In [ ]:
from transformers import pipeline
qa_model = pipeline("text2text-generation", model="google/flan-t5-base", device=0)

## Define a Retrieval-Augmented Generation (RAG) function

In [ ]:
def generate_answer(query):
    # Retrieve more document chunks for a broader context
    docs = vectorstore.similarity_search(query, k=6)
    # Filter out chunks that are very short (e.g., less than 20 words)
    informative_docs = [doc for doc in docs if len(doc.page_content.split()) > 20]
    context = " ".join([doc.page_content for doc in informative_docs])

    # Create a detailed prompt instructing the model to synthesize insights on performance
    prompt = (
        "Below are excerpts from JP Morgan earnings call transcripts. "
        "Please analyze these excerpts and provide a detailed summary of the key insights regarding quarterly performance. "
        "Include any performance metrics, trends, improvements or declines, and significant factors mentioned. "
        "If available, mention any specific challenges or notable observations about quarterly performance.\n\n"
        "Transcript Excerpts:\n"
        f"{context}\n\n"
        "Summary:"
    )

    # Optionally, increase max_length to allow a longer response
    result = qa_model(prompt, max_length=512, temperature=0.7)
    return result[0]['generated_text']

## Test the RAG function with an example query

In [ ]:
# Test the updated RAG function with your query
test_query = "What insights about quarterly performance are highlighted in the transcripts?"
print("RAG Answer:", generate_answer(test_query))

## Wrap the RAG function as a tool for a LangChain agent

In [ ]:
# IMPORTANT: Replace 'YOUR_HF_API_TOKEN' with your actual Hugging Face API token or set it as an environment variable.
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HF_API_TOKEN"  # <-- Replace with your token

from langchain_huggingface import HuggingFaceEndpoint
from langchain.agents import initialize_agent, Tool

# Initialize the LLM endpoint using the updated HuggingFaceEndpoint.
llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/google/flan-t5-base",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
    model_kwargs={"temperature": 0.7}
)

# Define a tool that wraps our RAG function.
def rag_tool(query: str) -> str:
    return generate_answer(query)

tools = [
    Tool(
        name="JP Morgan RAG",
        func=rag_tool,
        description="Answers questions using JP Morgan earnings transcripts as context."
    )
]

# Create the agent using the defined tool.
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Use the agent to answer a sample query.
agent_query = "Summarize the key performance trends mentioned in the JP Morgan earnings transcripts."
agent_answer = agent.run(agent_query)
print("Agent Answer:", agent_answer)

In [ ]:
"""
===================================================
Author: Sheldon Kemper
Role: Data Engineering Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/sheldon-kemper
Date: 2025-02-04
Version: 2.0

Description:
    This notebook implements a Retrieval-Augmented Generation (RAG) system using cleaned
    data from Bank of England projects. The data is stored in two CSV files – one containing the management
    discussion and one containing the questions and answers. The notebook loads these CSV files from Google Drive,
    converts each row into a Document object, and indexes them using a Chroma vector store with Sentence Transformer embeddings.
    A text generation model (Flan-T5) is then used to answer queries based on the retrieved context.
===================================================
"""

# Step 1: Mount Google Drive (if running in Google Colab)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Step 2: Import necessary libraries
import pandas as pd
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import pipeline

# Step 3: Define paths to your cleaned CSV files
management_path = "/content/drive/MyDrive/BOE/bank_of_england/data/processed/management_discussion.csv"
qa_path = "/content/drive/MyDrive/BOE/bank_of_england/data/processed/qa_section.csv"

# Load the CSV files
management_df = pd.read_csv(management_path)
qa_df = pd.read_csv(qa_path)

print("Management Discussion CSV shape:", management_df.shape)
print("Q&A CSV shape:", qa_df.shape)

# Step 4: Create Document objects from each CSV row.
# Assumption: Each CSV has a column named "content" that holds the cleaned text.
docs = []

# Process management discussion data
for index, row in management_df.iterrows():
    content = row["content"]  # adjust this if your column name is different
    docs.append(Document(page_content=content, metadata={"source": "management_discussion"}))

# Process Q&A data
for index, row in qa_df.iterrows():
    content = row["content"]
    docs.append(Document(page_content=content, metadata={"source": "qa_section"}))

print("Total documents created:", len(docs))

# Step 5: Initialize Sentence Transformer embeddings and create a Chroma vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(docs, embeddings, collection_name="boe_cleaned_data")
print("Chroma vector store created from cleaned BOE data.")

# Step 6: Load the text generation model (Flan-T5) using Hugging Face's transformers pipeline.
# Note: Ensure that your runtime is set to GPU for faster inference.
qa_model = pipeline("text2text-generation", model="google/flan-t5-base", device=0)

# Step 7: Define the Retrieval-Augmented Generation (RAG) function
def generate_answer(query):
    # Retrieve the top 6 most similar documents from the vector store
    retrieved_docs = vectorstore.similarity_search(query, k=6)
    # Combine the content from the retrieved documents
    context = " ".join([doc.page_content for doc in retrieved_docs])

    # Construct a detailed prompt instructing the model to summarize key performance insights
    prompt = (
        "Below are excerpts from the Bank of England management discussion and Q&A sections. "
        "Please analyze these excerpts and provide a detailed summary of the key insights regarding quarterly performance. "
        "Include performance metrics, trends, improvements or declines, and any notable observations.\n\n"
        "Excerpts:\n"
        f"{context}\n\n"
        "Summary:"
    )

    result = qa_model(prompt, max_length=512, temperature=0.7)
    return result[0]['generated_text']

# Step 8: Test the RAG function with a sample query
test_query = "What insights about quarterly performance are highlighted in the processed data?"
print("RAG Answer:", generate_answer(test_query))
